# Gera pdfs

## Carrega Pacotes

In [13]:
# Install a conda package in the current Jupyter kernel
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install xlrd

    100% |████████████████████████████████| 112kB 1.5MB/s ta 0:00:01


In [4]:
import pandas as pd
import re
from titlecase import titlecase

## Define Constantes

In [249]:

strlatex = """
  \\documentclass[twoside]{{article}}
  \\usepackage[affil-it]{{authblk}}
  \\usepackage{{lipsum}} % Package to generate dummy text throughout \
this template
  \\usepackage{{eurosym}}
  \\usepackage[sc]{{mathpazo}} % Use the Palatino font
  \\usepackage[T1]{{fontenc}} % Use 8-bit encoding that has 256 glyphs
  \\usepackage[utf8]{{inputenc}}
  \\linespread{{1.05}} % Line spacing-Palatino needs more space between \
lines
  \\usepackage{{microtype}} % Slightly tweak font spacing for \
aesthetics\[IndentingNewLine]
  \\usepackage[hmarginratio=1:1,top=32mm,columnsep=20pt]{{geometry}} % \
Document margins
  \\usepackage{{multicol}} % Used for the two-column layout of the \
document
  \\usepackage[hang,small,labelfont=bf,up,textfont=it,up]{{caption}} % \
Custom captions under//above floats in tables or figures
  \\usepackage{{booktabs}} % Horizontal rules in tables
  \\usepackage{{float}} % Required for tables and figures in the \
multi-column environment-they need to be placed in specific locations \
with the[H] (e.g. \\begin{{table}}[H])
  \\usepackage{{hyperref}} % For hyperlinks in the PDF
  \\usepackage{{lettrine}} % The lettrine is the first enlarged letter \
at the beginning of the text
  \\usepackage{{paralist}} % Used for the compactitem environment which \
makes bullet points with less space between them
  \\usepackage{{abstract}} % Allows abstract customization
  \\renewcommand{{\\abstractnamefont}}{{\\normalfont\\bfseries}} 
  %\\renewcommand{{\\abstracttextfont}}{{\\normalfont\\small\\itshape}} % \
Set the abstract itself to small italic text\[IndentingNewLine]
  \\usepackage{{titlesec}} % Allows customization of titles
  \\renewcommand\\thesection{{\\Roman{{section}}}} % Roman numerals for \
the sections
  \\renewcommand\\thesubsection{{\\Roman{{subsection}}}} % Roman numerals \
for subsections
  \\titleformat{{\\section}}[block]{{\\large\\scshape\\centering}}{{\\\
thesection.}}{{1em}}{{}} % Change the look of the section titles
  \\titleformat{{\\subsection}}[block]{{\\large}}{{\\thesubsection.}}{{1em}}{{}}\
 % Change the look of the section titles
  \\usepackage{{fancyhdr}} % Headers and footers
  \\pagestyle{{fancy}} % All pages have headers and footers
  \\fancyhead{{}} % Blank out the default header
  \\fancyfoot{{}} % Blank out the default footer
  \\fancyhead[C]{{X-meeting $\\bullet$ October 2019 $\\bullet$ Campos do  \
Jord\\~ao}} % Custom header text
  \\fancyfoot[RO,LE]{{}} % Custom footer text
  %----------------------------------------------------------------------------------------
\
  % TITLE SECTION
  %\
----------------------------------------------------------------------------------------\
 
 
 \\title{{\\vspace{{-15mm}}\\fontsize{{24pt}}{{10pt}}\\selectfont\\textbf{{ {} }}}} % Article title
  
  
  \\author{{ {} }}
  
  \\affil{{ {} }}
  \\vspace{{-5mm}}
  \\date{{}}
  
  %----------------------------------------------------------------------------------------\
 
  
  \\begin{{document}}
  
  
  \\maketitle % Insert title
  
  
  \\thispagestyle{{fancy}} % All pages have headers and footers\

  %----------------------------------------------------------------------------------------\
  
  % ABSTRACT
  
  %----------------------------------------------------------------------------------------\
  
  
  \\begin{{abstract}}
  {}
  
  Funding: {} \\\\ 
  \\end{{abstract}}
  \\end{{document}} """



strlatex2 = """\\procpaper[switch=45,
    title={{{}}}, 
    author={{{}}}, 
    index={{{{{}}}}}]
    {{{}}} \n\n """

replacements={"Î¦" : "$\\Phi$ ", "\'" : "'", "ﬁ" : "fi", "á" : "\\'a",
 "\[Gamma]" : "$\\gamma$", "ï\[Not]<81>" : "fi",
 "\[RightArrow]" : "$\\rightarrow$", " Î\.b3" : "$ \\gamma$",
 " â<86><92> â<88><9e>" : " $\\rightarrow$", "â<80>\.b2" : "'",
 "Â\[PlusMinus]" : "$\\pm$", "â<89>¤" : "$\\leq$",
 "Â\[Degree]" :  "$^o$", "\Âº" : "$^o$", " \â<84>¢ CL" : " ",
 "Â\.b4" : "'", "â<89>¥" : "$\\leq$", "â<80><8b>â" : " ", "¹" : "1",
 "â" : "\\^a", "\[OpenCurlyDoubleQuote]" : "``",
 "\[CloseCurlyDoubleQuote]" : "''", "ã" : "\\~a", "á" : "\\'a",
 "ê" : "\\^e", "é" : "\\'e", "õ" : "\\~o", "ó" : "\\'o",
 "ô" : "\\^o", "ú" : "\\'u", "ç" : "\\c{c}", "í" : "\\'{\\i}",
 "Ã" : "\\~A", "Á" : "\\'A", "Ê" : "\\^E", "É" : "\\'E",
 "Õ" : "\\~O", "Ó" : "\\'O", "Ô" : "\\^O", "Ú" : "\\'U",
 "Ç" : "\\c{C}", "Í" : "\\'I", "Å" : "$\\AA$", "ß" : "$\\beta$",
 "\[CloseCurlyQuote]" : "'", "\[OpenCurlyQuote]" : "'", "%" : "\\%",
  "ß" : "$\\beta$",  "_" : "\\_",         
              "±":"$\pm$",
              "´":"", "µ":"$\mu$",
              ",":", ",
              "&":"\&",
 "ö" : "\\\"o", "ü" : "\\\"u",
 "à" : "\\`a", "À" : "\\`A", "€": "\\euro",
 "®" : "\\textsuperscript{\\textcopyright}", "¾" : "3/4",
 "ñ" : "\\~n", "ï" : "\\\"{\\i}", "I'" : "\\'I", "#" : "\\#",
 "i\.b4" : "\\'{\\i}",
 " " : " ",  "°" : "$^o$"}
prefix=""

## Funções Gerais

In [250]:
def namecase(str):
    """Converte Strings para Capitalização Brasileira"""
    lissubs=[(" De "," de "),(" Do "," do "), (" Da ", " da "), (" Das ", " das "),
             (" Dos "," dos ")]
    str2=titlecase(str)
    for subs in lissubs:
        str2=str2.replace(subs[0],subs[1])
    return str2



In [251]:
def brnames(str):
    """Gera Nomes para a regra ABNT"""
    lis=str.split(" ")
    str2=lis[-1]+ ", "
    for word in lis[0:-1]:
        str2=str2+" "+word
    return str2

In [252]:
def orderedunion(lis):
    """Elimina Repetições e mantém a ordem"""
    lis2=[]
    for elem in lis:
        if not (elem in lis2):
            lis2.append(elem)
    return lis2

In [253]:


def fixbin(string,replacements):
    for k, v in replacements.items():
        string=string.replace(k, v)
    return string

## Funções Específicas


In [254]:
def geraAuthors(str):
    """Gera uma Lista de Autores"""
    m = re.findall('[^(),]+\([^()]+[^()]+\)', str)
    return [namecase(word.split("(")[0].strip()) for word in m]

def geraAfil(str):
    """Gera uma lista de Afiliações"""
    m = re.findall('\([^()]+[^()]+\)', str)
    return [inst[1:-1] for inst in m]

In [255]:
artsds["Autores"]

Número
180792    LETICIA FERREIRA LIMA,Rodrigo Jardim,Renata Sc...
180929    Arthur Sant'Anna Feltrin,Ana Carolina Tahira,S...
181353    Felipe Marques de Almeida,Georgios Joannis Pap...
186188    Luis Augusto Franco López,César Alberto Bravo ...
188839    Christian Reis Meneguin,Prof. Dr. David Corrêa...
188974    Hemanoel Passarelli Araujo,JUSSARA KASUKO PALM...
189373    Bruno Henrique Ribeiro da Fonseca,Douglas Silv...
189564    Bidossessi Wilfried Hounkpe,Fernando Ferreira ...
189566    Bidossessi Wilfried Hounkpe,Francine Chenou,Fr...
189628                   Aline Rodrigheri Ioste,Alan Durham
189638    Ondina Fonseca de Jesus Palmeira,Larissa Matos...
190006    Cinthia Caroline Alves,Eduardo Antônio Donadi,...
190030    Thaís Cristina Vilela Rodrigues,Arun Kumar Jai...
190037    Flavia Regina Florencio de Athayde,Flavia Lomb...
190043    Camilla Reginatto De Pierri,Diogo de Jesus Soa...
190044    MAURICIO DE ALVARENGA MUDADU,Adhemar Zerlotini...
190045    Diogo de Jesus Soares M

In [256]:
def geraStringAfilOld(lis):
    lisafil=orderedunion(lis)
    stringafil="" 
    i=1
    for inst in lisafil:
        stringafil=stringafil+str(i)+" "+inst+"\n\n"
        i=i+1
    return stringafil

def geraStringAfil(lis):
    return fixbin(record["Institution"],replacements)
    
def geraStringAuthorsOld(record):
    lisafil=record["Afil"]
    lisauthors=record["Authors"]
    dictauthors={}
    dictafil={}
    lisafilorder=orderedunion(lisafil)
    for i in range(len(lisauthors)):
        dictauthors[lisauthors[i]]=lisafil[i]
    for i in range(len(lisafilorder)):
        dictafil[lisafilorder[i]]=i
    strsaida=""
    for author in lisauthors:
        num=dictafil[dictauthors[author]]
        strsaida=strsaida+author+"$^{"+str(num+1)+"}$, "
    return strsaida[:-1]

def geraStringAuthors(record):
    return record["Autores"]

    

## Entrada dos Dados

In [257]:
artsds=pd.read_excel(prefix+'Relatorio de submissoes avaliadas_14-10-2019_14-58-12.xlsx',index_col=0)

In [258]:
artsds

,Data,Modalidade,Área Temática,Título,Autores,Emails,Apresentadores,Situação,Apresentado,Submeteu Trabalho Completo,Resumo,Link Trabalho Completo,Arquivo,Arquivo do Resumo,Funding,Institution
Número,,,,,,,,,,,,,,,,
180792,25/07/2019,Poster,Phylogeny and Evolution,TOLL receptor gene family evolution in insects,"LETICIA FERREIRA LIMA,Rodrigo Jardim,Renata Sc...","leticialimaf@yahoo.com.br,jardim.rodrigo@gmail...",LETICIA FERREIRA LIMA,Aprovado,Não,Não,Arthropoda can be found in almost every habita...,NaN,NaN,NaN,NaN,Laboratório de Biologia Computacional e Sistem...
180929,26/07/2019,HighLight Tracks,Systems Biology and Networks,Assessment of complementarity of WGCNA and NER...,"Arthur Sant'Anna Feltrin,Ana Carolina Tahira,S...","s.feltrin@ufabc.edu.br,tahira.ana@gmail.com,se...",Arthur Sant'Anna Feltrin,Aprovado,Não,Não,Schizophrenia is a complex disorder that invol...,NaN,https://even3.blob.core.windows.net/formulario...,NaN,NaN,Universidade Federal do ABC
181353,29/07/2019,Poster,Genes and Genomics,A container-based pipeline for bacterial genom...,"Felipe Marques de Almeida,Georgios Joannis Pap...","felipemarques89@gmail.com,gpappas@unb.br",Felipe Marques de Almeida,Aprovado,Não,Não,Advances in DNA sequencing technologies are re...,NaN,NaN,NaN,Conselho Nacional de Desenvolvimento Científic...,Universidade de Brasília
186188,01/08/2019,Poster,Systems Biology and Networks,Reconstruction of metabolic pathways of Klebsi...,"Luis Augusto Franco López,César Alberto Bravo ...","augustofranco93@gmail.com,cabpariente@uesc.br",Luis Augusto Franco López,Aprovado,Não,Não,Pests are known to cause significant damage to...,NaN,NaN,NaN,NaN,"Universidade Estadual de Santa Cruz, Ilhéus, B..."
188839,09/08/2019,Prêmio de Teses e Dissertações,Database and Software Development,SELEÇÃO DE CARACTERÍSTICAS A PARTIR DA INTEGRA...,"Christian Reis Meneguin,Prof. Dr. David Corrêa...","chrisme@gmail.com,dbvmartins@yahoo.com.br",Christian Reis Meneguin,Não Aprovado,Não,Não,Complex diseases are usually consequences of i...,NaN,NaN,https://even3.blob.core.windows.net/formulario...,NaN,Universidade Federal do ABC - UFABC
188974,09/08/2019,HighLight Tracks,Genes and Genomics,Genomic analysis unveils important aspects of ...,"Hemanoel Passarelli Araujo,JUSSARA KASUKO PALM...","hemanuel.passarelli@gmail.com,jukasuko@gmail.c...",Hemanoel Passarelli Araujo,Aprovado,Não,Não,Klebsiella aerogenes is an important pathogen ...,NaN,https://even3.blob.core.windows.net/formulario...,NaN,NaN,NaN
189373,11/08/2019,Prêmio de Teses e Dissertações,Database and Software Development,UNRAVELING MIRTRONS KNOWLEDGE WITH DATA MINING...,"Bruno Henrique Ribeiro da Fonseca,Douglas Silv...","fonseca.adm@hotmail.com,doug@rc.unesp.br,pasch...",Bruno Henrique Ribeiro da Fonseca,Não Aprovado,Não,Não,"In the current literature, there was no rep...",NaN,NaN,https://even3.blob.core.windows.net/formulario...,NaN,UTFPR
189564,12/08/2019,Poster,RNA and Transcriptomics,Correlation of shared transcriptomic signature...,"Bidossessi Wilfried Hounkpe,Fernando Ferreira ...","bidossessi1@live.fr,ferreira@unicamp.br,erich@...",Bidossessi Wilfried Hounkpe,Aprovado,Não,Não,While ischemia-reperfusion injury (IRI) is wid...,NaN,NaN,NaN,FAPESP grants # 2015/24666-3; CNPq Brazil. gra...,"Faculty of Medical Sciences, Unicamp"
189566,12/08/2019,Poster,Database and Software Development,HT Atlas v1.0 database: redefining human and m...,"Bidossessi Wilfried Hounkpe,Francine Chenou,Fr...","bidossessi1@live.fr,franche.2008@hotmail.fr,fr...",Bidossessi Wilfried Hounkpe,Aprovado,Não,Não,Housekeeping (HK) genes are constitutively exp...,NaN,NaN,NaN,"FAPESP grants # 2016/14172-6, 2015/24666-3; C...","Faculty of Medical Sciences, Unicamp"


Insere duas colunas no lugar da Coluna de Autores

In [259]:
artsds.columns

Index(['Data', 'Modalidade', 'Área Temática', 'Título', 'Autores', 'Emails',
       'Apresentadores', 'Situação', 'Apresentado',
       'Submeteu Trabalho Completo', 'Resumo', 'Link Trabalho Completo',
       'Arquivo', 'Arquivo do Resumo', 'Funding', 'Institution'],
      dtype='object')

In [260]:
artsds["Autores"]

Número
180792    LETICIA FERREIRA LIMA,Rodrigo Jardim,Renata Sc...
180929    Arthur Sant'Anna Feltrin,Ana Carolina Tahira,S...
181353    Felipe Marques de Almeida,Georgios Joannis Pap...
186188    Luis Augusto Franco López,César Alberto Bravo ...
188839    Christian Reis Meneguin,Prof. Dr. David Corrêa...
188974    Hemanoel Passarelli Araujo,JUSSARA KASUKO PALM...
189373    Bruno Henrique Ribeiro da Fonseca,Douglas Silv...
189564    Bidossessi Wilfried Hounkpe,Fernando Ferreira ...
189566    Bidossessi Wilfried Hounkpe,Francine Chenou,Fr...
189628                   Aline Rodrigheri Ioste,Alan Durham
189638    Ondina Fonseca de Jesus Palmeira,Larissa Matos...
190006    Cinthia Caroline Alves,Eduardo Antônio Donadi,...
190030    Thaís Cristina Vilela Rodrigues,Arun Kumar Jai...
190037    Flavia Regina Florencio de Athayde,Flavia Lomb...
190043    Camilla Reginatto De Pierri,Diogo de Jesus Soa...
190044    MAURICIO DE ALVARENGA MUDADU,Adhemar Zerlotini...
190045    Diogo de Jesus Soares M

## Separação em Grupos

In [261]:
artsdsgrouped=artsds.groupby(["Modalidade"],axis=0)
artsdsposter=artsdsgrouped.get_group('Poster')
artsdsHT=artsdsgrouped.get_group('HighLight Tracks')


In [262]:
artsds.iloc[0].index

Index(['Data', 'Modalidade', 'Área Temática', 'Título', 'Autores', 'Emails',
       'Apresentadores', 'Situação', 'Apresentado',
       'Submeteu Trabalho Completo', 'Resumo', 'Link Trabalho Completo',
       'Arquivo', 'Arquivo do Resumo', 'Funding', 'Institution'],
      dtype='object')

## Gera os arquivos com os abstracts

In [263]:
for index, record in artsds.iterrows():
    author=fixbin(record["Autores"],replacements)
    title=fixbin(record["Título"],replacements)
    if record["Institution"]!=record["Institution"]:
        afiliation=""
    else:
        afiliation=fixbin(record["Institution"],replacements)
    print(index)
    if record["Modalidade"]=="Poster":
        abstract=fixbin(record["Resumo"],replacements)
    else:
        abstract=""
    if record["Funding"]!=record["Funding"]:
        funding=""
    else:
        funding=fixbin(record["Funding"],replacements)
    stringfileresumo=strlatex.format(title,author,afiliation,abstract,funding)
    f = open('papers/art'+str(index)+".tex", 'w')
    f.write(stringfileresumo)
    f.close()

180792
180929
181353
186188
188839
188974
189373
189564
189566
189628
189638
190006
190030
190037
190043
190044
190045
190060
190102
190224
190227
190251
190288
190393
190394
190542
190633
190656
190658
190830
190847
190850
190910
190947
191455
191535
192650
192871
193434
193435
194221
194642
195426
195814
196113
196221
196669
197514
197831
198393
198401
198424
199094
199174
199175
199542
200200
200264
200294
200803
200819
200863
201173
201185
201202
201223
201248
201254
201276
201317
201327
201328
201343
201344
201350
201354
201371
201380
201385
201421
201444
201447
201454
201465
201492
201494
201507
201515
201518
201545
201547
201568
201570
201573
201577
201584
201585
201598
201610
201613
201623
201628
201630
201634
201635
201636
201638
201643
201645
201650
201655
201658
201660
201661
201666
201671
201674
201688
201692
201699
201712
201719
201722
201728
201730
201734
201740
201772
201829
201860
201930
201942
202000
202003
202006
202028
202034
202045
202076
202192
202805
208811
208813

In [264]:
!pwd

/home/lemke/Documents/procxmeeting2019


## Gera os arquivos com os índices 

In [265]:
areas = sorted(list(set(artsdsposter["Área Temática"])))
groupareas=artsdsposter.groupby(["Área Temática"],axis=0)
compstr = "";
for area in areas:
    print(area)
    ds=groupareas.get_group(area)
#    ds=ds.sort_values("Poster Code")
    sessionstr = "\\chapter{" + area + "}\n";
    for index, record in ds.iterrows():
        title=fixbin(record["Título"],replacements)
        author=record["Autores"]
        file="art"+str(index)
        indexstr=",".join(["\\index{"+brnames(aut)+"}" for aut in record["Autores"].split(",")])
        sessionstr=sessionstr+strlatex2.format(title,author,indexstr,file)
    compstr = compstr + sessionstr
f = open('posters.tex', 'w')
f.write(compstr)
f.close()

Database and Software Development
Genes and Genomics
Phylogeny and Evolution
Proteins and Proteomics
RNA and Transcriptomics
Systems Biology and Networks


In [266]:
compstr = "";
ds=artsdsHT
sessionstr=""
#ds=ds.sort_values("Poster Code")
for index, record in ds.iterrows():
    print(index)
    title=fixbin(record["Título"],replacements)
    author=record["Autores"]
    file="art"+str(index)
    indexstr=",".join(["\\index{"+brnames(aut)+"}" for aut in record["Autores"].split(",")])
    sessionstr=sessionstr+strlatex2.format(title,author,indexstr,file)
compstr = compstr + sessionstr
f = open('papers.tex', 'w')
f.write(compstr)
f.close()

180929
188974
190030
190542
190830
201371
201545


In [267]:
artsdsHT


,Data,Modalidade,Área Temática,Título,Autores,Emails,Apresentadores,Situação,Apresentado,Submeteu Trabalho Completo,Resumo,Link Trabalho Completo,Arquivo,Arquivo do Resumo,Funding,Institution
Número,,,,,,,,,,,,,,,,
180929,26/07/2019,HighLight Tracks,Systems Biology and Networks,Assessment of complementarity of WGCNA and NER...,"Arthur Sant'Anna Feltrin,Ana Carolina Tahira,S...","s.feltrin@ufabc.edu.br,tahira.ana@gmail.com,se...",Arthur Sant'Anna Feltrin,Aprovado,Não,Não,Schizophrenia is a complex disorder that invol...,NaN,https://even3.blob.core.windows.net/formulario...,NaN,NaN,Universidade Federal do ABC
188974,09/08/2019,HighLight Tracks,Genes and Genomics,Genomic analysis unveils important aspects of ...,"Hemanoel Passarelli Araujo,JUSSARA KASUKO PALM...","hemanuel.passarelli@gmail.com,jukasuko@gmail.c...",Hemanoel Passarelli Araujo,Aprovado,Não,Não,Klebsiella aerogenes is an important pathogen ...,NaN,https://even3.blob.core.windows.net/formulario...,NaN,NaN,NaN
190030,13/08/2019,HighLight Tracks,Genes and Genomics,Reverse vaccinology and subtractive genomics r...,"Thaís Cristina Vilela Rodrigues,Arun Kumar Jai...","thais-cvr@hotmail.com,arunjaiswal1411@gmail.co...",Thaís Cristina Vilela Rodrigues,Aprovado,Não,Não,Pneumonia is an infectious disease caused by b...,NaN,https://even3.blob.core.windows.net/formulario...,NaN,NaN,Universidade Federal do Triângulo Mineiro
190542,14/08/2019,HighLight Tracks,Proteins and Proteomics,nAPOLI: a graph-based strategy to detect and v...,"Alexandre Victor Fassio,Lucianna Helene Silva ...","alexandrefassiogcc@gmail.com,luciannahss@gmail...",Alexandre Victor Fassio,Aprovado,Não,Não,"In the manuscript, we proposed nAPOLI (Analysi...",NaN,https://even3.blob.core.windows.net/formulario...,NaN,NaN,"Universidade Federal de Viçosa, UFV"
190830,15/08/2019,HighLight Tracks,Database and Software Development,mirtronDB: a mirtron knowledge base,"Bruno Henrique Ribeiro da Fonseca,Douglas Silv...","fonseca.adm@hotmail.com,doug@rc.unesp.br,pasch...",Alexandre R Paschoal,Aprovado,Não,Não,Mirtrons arise from short introns with atypica...,NaN,https://even3.blob.core.windows.net/formulario...,NaN,NaN,UTFPR
201371,12/09/2019,HighLight Tracks,Genes and Genomics,Prediction of new vaccine targets in the core ...,"Carlos Leonardo Araújo,Jorianne Thyeska Castro...","leonardo.a.araujo@hotmail.com,joriannealves@gm...",Wylerson Nogueira,Aprovado,Não,Não,Corynebacterium pseudotuberculosis is the etio...,NaN,https://even3.blob.core.windows.net/formulario...,NaN,NaN,Universidade Federal do Pará
201545,12/09/2019,HighLight Tracks,RNA and Transcriptomics,The pH Signaling Transcription Factor PAC-3 Re...,"Maíra Pompeu Martins,Nilce M. Martinez-Rossi,P...","mairapompeu@gmail.com,nmmrossi@usp.br,psanches...",Maíra Pompeu Martins,Revisão necessária,Não,Não,Advances in the understanding of functional ge...,NaN,https://even3.blob.core.windows.net/formulario...,NaN,NaN,NaN
